In [ ]:
!pip install torch torchvision torchaudio
!pip install dlib opencv-python pandas tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile

zip_path = "/content/drive/MyDrive/TTI_results.zip"
extract_to = "/content/TTI_results"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)


In [ ]:
import os


images_dir = "/content/TTI_results/TTI_results/generated_images"

for fname in os.listdir(images_dir):
    full_path = os.path.join(images_dir, fname)
    if os.path.isdir(full_path):
        continue

    # Add .png if the file has no extension
    name, ext = os.path.splitext(fname)
    if ext == "":
        new_name = fname + ".png"
        new_path = os.path.join(images_dir, new_name)
        os.rename(full_path, new_path)
        print(f"Renamed: {fname} → {new_name}")

original_paths = sorted([
    os.path.join(images_dir, fname)
    for fname in os.listdir(images_dir)
    if "_original" in fname.lower() and fname.lower().endswith((".png", ".jpg"))
])

enhanced_paths = sorted([
    os.path.join(images_dir, fname)
    for fname in os.listdir(images_dir)
    if "_enhanced" in fname.lower() and fname.lower().endswith((".png", ".jpg"))
])

print(f"\nOriginal images found: {len(original_paths)}")
print(f"Enhanced images found: {len(enhanced_paths)}")

print("\nSample originals:", original_paths[:3])
print("Sample enhanced:", enhanced_paths[:3])


Renamed: sample_357_enhanced → sample_357_enhanced.png
Renamed: sample_606_original → sample_606_original.png
Renamed: sample_795_enhanced → sample_795_enhanced.png
Renamed: sample_637_enhanced → sample_637_enhanced.png
Renamed: sample_789_enhanced → sample_789_enhanced.png
Renamed: sample_619_original → sample_619_original.png
Renamed: sample_717_original → sample_717_original.png
Renamed: sample_784_enhanced → sample_784_enhanced.png
Renamed: sample_182_original → sample_182_original.png
Renamed: sample_906_original → sample_906_original.png
Renamed: sample_329_enhanced → sample_329_enhanced.png
Renamed: sample_065_original → sample_065_original.png
Renamed: sample_351_original → sample_351_original.png
Renamed: sample_672_enhanced → sample_672_enhanced.png
Renamed: sample_900_enhanced → sample_900_enhanced.png
Renamed: sample_287_enhanced → sample_287_enhanced.png
Renamed: sample_940_original → sample_940_original.png
Renamed: sample_401_original → sample_401_original.png
Renamed: s

In [ ]:
BASE_DIR = "/content/metrics_results"

# Pre-downloaded models
FAIRFACE_MODEL_DIR = os.path.join(BASE_DIR, "fair_face_model")
DLIB_MODEL_DIR = os.path.join(BASE_DIR, "dlib_models")

# Runtime folders & outputs
INPUT_DIR = images_dir
DETECTED_DIR = os.path.join(BASE_DIR, "detected_faces")
OUTPUT_CSV = os.path.join(BASE_DIR, "test_outputs.csv")
BIAS_W_CSV = os.path.join(BASE_DIR, "bias_w_metrics.csv")
BIAS_P_CSV = os.path.join(BASE_DIR, "bias_p_metrics.csv")
ENS_CSV = os.path.join(BASE_DIR, "ens.csv")
KL_CSV = os.path.join(BASE_DIR, "kl_divergence_metrics.csv")

os.makedirs(DETECTED_DIR, exist_ok=True)

In [ ]:
import zipfile
import pandas as pd
import torch
import torch.nn as nn
import numpy as np
import torchvision
from torchvision import transforms
import dlib

# Utility to convert dlib rectangle to bounding box
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

# Ensure directory exists
def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Softmax helper
def softmax(x):
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

In [ ]:
# Detect faces and save aligned images
def detect_face(image_paths, SAVE_DETECTED_AT, default_max_size=800, size=300, padding=0.25):
    face_detector = dlib.get_frontal_face_detector()
    sp = dlib.shape_predictor(os.path.join(DLIB_MODEL_DIR, "shape_predictor_5_face_landmarks.dat"))

    for index, image_path in enumerate(image_paths):
        if index % 100 == 0:
            print('Processing image: %d/%d' % (index, len(image_paths)))
        img = dlib.load_rgb_image(image_path)

        old_height, old_width, _ = img.shape
        if old_width > old_height:
            new_width, new_height = default_max_size, int(default_max_size * old_height / old_width)
        else:
            new_width, new_height = int(default_max_size * old_width / old_height), default_max_size

        img = dlib.resize_image(img, rows=new_height, cols=new_width)

        dets = face_detector(img, 1)
        if len(dets) == 0:
            print("No face detected in: {}".format(image_path))
            continue

        faces = dlib.full_object_detections()
        for rect in dets:
            faces.append(sp(img, rect))

        images = dlib.get_face_chips(img, faces, size=size, padding=padding)
        for idx, image in enumerate(images):
            img_name = os.path.basename(image_path)
            base, ext = os.path.splitext(img_name)
            face_name = os.path.join(SAVE_DETECTED_AT, f"{base}_face{idx}{ext}")
            dlib.save_image(image, face_name)

In [ ]:
def is_image_file(path):
    if not os.path.isfile(path):
        return False
    _, ext = os.path.splitext(path.lower())
    return ext in {'.jpg', '.jpeg', '.png', '.bmp'}

def predict_age_gender_race(save_prediction_at, imgs_path='detected_faces/'):
    # collect only real image files
    img_names = []
    for entry in os.listdir(imgs_path):
        full = os.path.join(imgs_path, entry)
        if is_image_file(full):
            img_names.append(full)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

    # Load pretrained models
    model_fair_7 = torchvision.models.resnet34(pretrained=True)
    model_fair_7.fc = nn.Linear(model_fair_7.fc.in_features, 18)
    model_fair_7.load_state_dict(
    torch.load(os.path.join(FAIRFACE_MODEL_DIR, "res34_fair_align_multi_7_20190809.pt"), map_location=device))

    model_fair_7 = model_fair_7.to(device).eval()

    # Image transforms
    trans = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

    face_names = []
    race_preds_fair, gender_preds_fair, age_preds_fair = [], [], []
    race_scores_fair, gender_scores_fair, age_scores_fair = [], [], []

    for index, img_name in enumerate(img_names):
        if index % 100 == 0:
            print(f"Predicting: {index}/{len(img_names)}")

        # Defensive load
        try:
            image_raw = dlib.load_rgb_image(img_name)
        except Exception as e:
            print(f"Warning: skipping {img_name} (load error: {e})")
            continue

        # proceed
        face_names.append(img_name)
        image = trans(image_raw).view(1, 3, 224, 224).to(device)

        # fair 7 model
        outputs = model_fair_7(image).cpu().detach().numpy().squeeze()
        race_score = softmax(outputs[:7])
        gender_score = softmax(outputs[7:9])
        age_score = softmax(outputs[9:18])

        race_preds_fair.append(np.argmax(race_score))
        gender_preds_fair.append(np.argmax(gender_score))
        age_preds_fair.append(np.argmax(age_score))
        race_scores_fair.append(race_score)
        gender_scores_fair.append(gender_score)
        age_scores_fair.append(age_score)

    # Create dataframe
    result = pd.DataFrame({
        'face_name_align': face_names,
        'race_preds_fair': race_preds_fair,
        # 'race_preds_fair_4': race_preds_fair_4,
        'gender_preds_fair': gender_preds_fair,
        'age_preds_fair': age_preds_fair,
        'race_scores_fair': race_scores_fair,
        # 'race_scores_fair_4': race_scores_fair_4,
        'gender_scores_fair': gender_scores_fair,
        'age_scores_fair': age_scores_fair
    })

    # Decode labels
    race_labels_7 = ['White', 'Black', 'Latino_Hispanic', 'East Asian', 'Southeast Asian', 'Indian', 'Middle Eastern']
    race_labels_4 = ['White', 'Black', 'Asian', 'Indian']
    gender_labels = ['Male', 'Female']
    age_labels = ['0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']

    result['race'] = result['race_preds_fair'].apply(lambda x: race_labels_7[x])
    # result['race4'] = result['race_preds_fair_4'].apply(lambda x: race_labels_4[x])
    result['gender'] = result['gender_preds_fair'].apply(lambda x: gender_labels[x])
    result['age'] = result['age_preds_fair'].apply(lambda x: age_labels[x])

    # Drop raw prediction indices
    result = result.drop(columns=[
        'race_preds_fair',
        # 'race_preds_fair_4',
        'gender_preds_fair',
        'age_preds_fair'
    ])

    # Save cleaned result
    result.to_csv(save_prediction_at, index=False)
    print("Results saved to:", save_prediction_at)

In [ ]:
race_labels = ['White', 'Black', 'Latino_Hispanic', 'East Asian', 'Southeast Asian', 'Indian', 'Middle Eastern']
gender_labels = ['Male', 'Female']
age_labels = ['0-2', '3-9', '10-19', '20-29', '30-39', '40-49', '50-59', '60-69', '70+']

attributes_dict = {
    'race': race_labels,
    'gender': gender_labels,
    'age': age_labels
}

# Define single attributes and combinations to analyze
combinations_to_analyze = [
    ['race'], ['gender'], ['age'],
    ['race', 'gender'], ['race', 'age'], ['gender', 'age'],
    ['race', 'gender', 'age']
]

In [ ]:
def calculate_combined_bias_metrics(prediction_csv_path, bias_w_csv_path='bias_w_metrics.csv', bias_p_csv_path='bias_p_metrics.csv'):
    """
    Calculates Bias-W and Bias-P metrics for single and combined attributes
    and saves them to separate CSV files.
    """
    try:
        df = pd.read_csv(prediction_csv_path)
    except FileNotFoundError:
        print(f"Error: The file {prediction_csv_path} was not found.")
        return

    # Extract original image name from the face_name_align column
    df['original_image'] = df['face_name_align'].apply(lambda x: os.path.basename(x).split('_face')[0])

    bias_w_records = []
    bias_p_records = []

    for attribute_group in combinations_to_analyze:
        attr_group_name = '_'.join(attribute_group)

        # --- Calculate Bias-W (across overall results) ---
        na = 1
        for attr in attribute_group:
            na *= len(attributes_dict[attr])

        if len(attribute_group) == 1:
            attr_type = attribute_group[0]
            freq_w = df[attr_type].value_counts(normalize=True)
        else:
            freq_w = df.groupby(attribute_group).size() / len(df)

        bias_w_sum = 0

        # We need to iterate through all possible combinations for the sum
        if len(attribute_group) == 1:
            all_combs = attributes_dict[attribute_group[0]]
        elif len(attribute_group) == 2:
            all_combs = [(a, b) for a in attributes_dict[attribute_group[0]] for b in attributes_dict[attribute_group[1]]]
        else:
            all_combs = [(a, b, c) for a in attributes_dict[attribute_group[0]] for b in attributes_dict[attribute_group[1]] for c in attributes_dict[attribute_group[2]]]

        for comb in all_combs:
            if isinstance(comb, str):
                freq_a_w = freq_w.get(comb, 0)
            else:
                try:
                    freq_a_w = freq_w.loc[comb]
                except KeyError:
                    freq_a_w = 0
            bias_w_sum += (freq_a_w - (1 / na)) ** 2

        bias_w = np.sqrt((1 / na) * bias_w_sum)
        bias_w_records.append({
            'Attribute_Group': attr_group_name,
            'Bias-W': bias_w
        })

        # --- Calculate Bias-P (within each multi-face image) ---
        for image_name, group in df.groupby('original_image'):
            if len(group) > 1:
                if len(attribute_group) == 1:
                    attr_type = attribute_group[0]
                    freq_p = group[attr_type].value_counts(normalize=True)
                else:
                    freq_p = group.groupby(attribute_group).size() / len(group)

                bias_p_sum = 0
                for comb in all_combs:
                    if isinstance(comb, str):
                        freq_a_p = freq_p.get(comb, 0)
                    else:
                        try:
                            freq_a_p = freq_p.loc[comb]
                        except KeyError:
                            freq_a_p = 0
                    bias_p_sum += (freq_a_p - (1 / na)) ** 2

                bias_p = np.sqrt((1 / na) * bias_p_sum)

                bias_p_records.append({
                    'Original_Image': image_name,
                    'Attribute_Group': attr_group_name,
                    'Bias-P': bias_p
                })

    bias_w_df = pd.DataFrame(bias_w_records)
    bias_w_df.to_csv(bias_w_csv_path, index=False)
    print(f"Bias-W metrics saved to: {bias_w_csv_path}")

    bias_p_df = pd.DataFrame(bias_p_records)
    bias_p_df.to_csv(bias_p_csv_path, index=False)
    print(f"Bias-P metrics saved to: {bias_p_csv_path}")

In [ ]:
from itertools import product

def calculate_ens_metrics(prediction_csv_path, output_csv='ens_metrics.csv'):
    """
    Calculates the Effective Number of Species (ENS) for the specified attributes
    and their combinations.

    ENS = exp(- Σ_g p_g * ln p_g), where p_g are the group proportions.
    """
    try:
        df = pd.read_csv(prediction_csv_path)
    except FileNotFoundError:
        print(f"Error: The file {prediction_csv_path} was not found.")
        return

    ens_records = []

    for attrs in combinations_to_analyze:
        attrs = list(attrs)
        # Consider only rows where all attributes in the combination are present
        mask = df[attrs].notna().all(axis=1)
        sub = df.loc[mask, attrs].copy()

        denom = len(sub)
        if denom == 0:
            ens_records.append({
                'Attribute': "+".join(attrs),
                'ENS': 0.0
            })
            continue

        label_universes = [attributes_dict[a] for a in attrs]
        all_groups = list(product(*label_universes))

        if len(attrs) == 1:
            counts = sub[attrs[0]].value_counts().to_dict()
            p_list = []
            for lab in attributes_dict[attrs[0]]:
                c = counts.get(lab, 0)
                p_list.append(c / denom)
        else:
            counts = sub.value_counts().to_dict()
            p_list = []
            for grp in all_groups:
                c = counts.get(tuple(grp), 0)
                p_list.append(c / denom)

        # Compute Σ p_g ln p_g (ignore p_g = 0 terms)
        sum_p_ln_p = 0.0
        for p_g in p_list:
            if p_g > 0:
                sum_p_ln_p += p_g * np.log(p_g)

        ens_value = float(np.exp(-sum_p_ln_p))

        ens_records.append({
            'Attribute': "+".join(attrs),
            'ENS': ens_value
        })

    ens_df = pd.DataFrame(ens_records)
    ens_df.to_csv(output_csv, index=False)
    print(f"ENS metrics saved to: {output_csv}")


In [ ]:
import itertools

def calculate_kl_divergence(
    prediction_csv_path,
    output_csv='kl_divergence_metrics.csv',
    reference_distribution=None
):
    """
    Calculates KL Divergence for race, gender, age, and intersectional combinations.

    Parameters:
    - prediction_csv_path: CSV file with columns ['race', 'gender', 'age'].
    - output_csv: Path to save results.
    - reference_distribution: dict (optional) specifying reference distributions for
      each attribute or combination. Example:
        {
            'race': {'White': 0.4, 'Black': 0.3, ...},
            'race_gender': {('White','Male'): 0.2, ('White','Female'): 0.2, ...}
        }
      If None, assumes uniform distribution for each case.
    """
    try:
        df = pd.read_csv(prediction_csv_path)
    except FileNotFoundError:
        print(f"Error: The file {prediction_csv_path} was not found.")
        return

    kl_records = []

    for attribute_group in combinations_to_analyze:
        attr_group_name = '_'.join(attribute_group)

        # Generated distribution
        if len(attribute_group) == 1:
            gen_dist = df[attribute_group[0]].value_counts(normalize=True)
        else:
            gen_dist = df.groupby(attribute_group).size() / len(df)

        # Reference distribution
        if reference_distribution and attr_group_name in reference_distribution:
            ref_dist = pd.Series(reference_distribution[attr_group_name])
        else:
            # Uniform distribution
            if len(attribute_group) == 1:
                labels = attributes_dict[attribute_group[0]]
            else:
                labels = list(itertools.product(*(attributes_dict[attr] for attr in attribute_group)))
            ref_dist = pd.Series({label: 1/len(labels) for label in labels})

        all_labels = sorted(set(gen_dist.index) | set(ref_dist.index))
        gen_probs = np.array([gen_dist.get(label, 0) for label in all_labels], dtype=float)
        ref_probs = np.array([ref_dist.get(label, 0) for label in all_labels], dtype=float)

        # Clip to avoid log(0)
        eps = 1e-12
        gen_probs = np.clip(gen_probs, eps, 1)
        ref_probs = np.clip(ref_probs, eps, 1)

        # KL Divergence
        kl_value = np.sum(gen_probs * np.log(gen_probs / ref_probs))

        kl_records.append({
            'Attribute_Group': attr_group_name,
            'KL_Divergence': kl_value
        })

    kl_df = pd.DataFrame(kl_records)
    kl_df.to_csv(output_csv, index=False)
    print(f"KL Divergence metrics saved to: {output_csv}")


In [ ]:
def process_drive_and_predict(
    input_dir=INPUT_DIR,
    output_csv=OUTPUT_CSV,
    bias_p_csv=BIAS_P_CSV,
    bias_w_csv=BIAS_W_CSV,
    ens_csv=ENS_CSV,
    kl_csv=KL_CSV
):
    ensure_dir(DETECTED_DIR)

    print("Reading images from Google Drive...")

    img_paths = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp')):
                print("processing image: ", file.lower())
                img_paths.append(os.path.join(root, file))

    # Save image paths CSV
    input_csv_path = os.path.join(BASE_DIR, "image_paths.csv")
    pd.DataFrame({'img_path': img_paths}).to_csv(input_csv_path, index=False)

    # Run detection + predictions + metrics
    detect_face(img_paths, DETECTED_DIR)
    predict_age_gender_race(output_csv, DETECTED_DIR)

    calculate_combined_bias_metrics(output_csv, bias_w_csv, bias_p_csv)
    calculate_ens_metrics(output_csv, ens_csv)
    calculate_kl_divergence(output_csv, kl_csv)


In [ ]:
# === New: run pipeline separately for originals and enhanced ===

def process_paths_and_predict(img_paths, tag):
    """
    Minimal wrapper to reuse existing logic while separating outputs per image set.
    - img_paths: list of file paths to images
    - tag: short suffix like "original" or "enhanced"
    This preserves the core logic but changes only I/O destinations.
    """
    detected_dir = os.path.join(BASE_DIR, f"detected_faces_{tag}")
    ensure_dir(detected_dir)

    output_csv = os.path.join(BASE_DIR, f"test_outputs_{tag}.csv")
    bias_p_csv = os.path.join(BASE_DIR, f"bias_p_metrics_{tag}.csv")
    bias_w_csv = os.path.join(BASE_DIR, f"bias_w_metrics_{tag}.csv")  # will be deleted right after creation
    ens_csv = os.path.join(BASE_DIR, f"ens_{tag}.csv")
    kl_csv = os.path.join(BASE_DIR, f"kl_divergence_metrics_{tag}.csv")

    # Save the list of image paths used for this run (optional but handy for audit)
    img_list_csv = os.path.join(BASE_DIR, f"image_paths_{tag}.csv")
    pd.DataFrame({'img_path': img_paths}).to_csv(img_list_csv, index=False)

    # Run pipeline (reusing existing functions)
    detect_face(img_paths, detected_dir)
    predict_age_gender_race(output_csv, detected_dir)

    # Save metrics
    calculate_combined_bias_metrics(output_csv, bias_w_csv, bias_p_csv)
    calculate_ens_metrics(output_csv, ens_csv)
    calculate_kl_divergence(output_csv, kl_csv)

# Now run for both sets:
process_paths_and_predict(original_paths, "original")
process_paths_and_predict(enhanced_paths, "enhanced")


Processing image: 0/1000
Processing image: 100/1000
Processing image: 200/1000
Processing image: 300/1000
Processing image: 400/1000
Processing image: 500/1000
Processing image: 600/1000
Processing image: 700/1000
Processing image: 800/1000
Processing image: 900/1000
No face detected in: /content/TTI_results/TTI_results/generated_images/sample_903_original.png


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth


100%|██████████| 83.3M/83.3M [00:00<00:00, 173MB/s]


Predicting: 0/3978
Predicting: 100/3978
Predicting: 200/3978
Predicting: 300/3978
Predicting: 400/3978
Predicting: 500/3978
Predicting: 600/3978
Predicting: 700/3978
Predicting: 800/3978
Predicting: 900/3978
Predicting: 1000/3978
Predicting: 1100/3978
Predicting: 1200/3978
Predicting: 1300/3978
Predicting: 1400/3978
Predicting: 1500/3978
Predicting: 1600/3978
Predicting: 1700/3978
Predicting: 1800/3978
Predicting: 1900/3978
Predicting: 2000/3978
Predicting: 2100/3978
Predicting: 2200/3978
Predicting: 2300/3978
Predicting: 2400/3978
Predicting: 2500/3978
Predicting: 2600/3978
Predicting: 2700/3978
Predicting: 2800/3978
Predicting: 2900/3978
Predicting: 3000/3978
Predicting: 3100/3978
Predicting: 3200/3978
Predicting: 3300/3978
Predicting: 3400/3978
Predicting: 3500/3978
Predicting: 3600/3978
Predicting: 3700/3978
Predicting: 3800/3978
Predicting: 3900/3978
Results saved to: /content/metrics_results/test_outputs_original.csv
Bias-W metrics saved to: /content/metrics_results/bias_w_metric

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet34_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet34_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Predicting: 0/4441
Predicting: 100/4441
Predicting: 200/4441
Predicting: 300/4441
Predicting: 400/4441
Predicting: 500/4441
Predicting: 600/4441
Predicting: 700/4441
Predicting: 800/4441
Predicting: 900/4441
Predicting: 1000/4441
Predicting: 1100/4441
Predicting: 1200/4441
Predicting: 1300/4441
Predicting: 1400/4441
Predicting: 1500/4441
Predicting: 1600/4441
Predicting: 1700/4441
Predicting: 1800/4441
Predicting: 1900/4441
Predicting: 2000/4441
Predicting: 2100/4441
Predicting: 2200/4441
Predicting: 2300/4441
Predicting: 2400/4441
Predicting: 2500/4441
Predicting: 2600/4441
Predicting: 2700/4441
Predicting: 2800/4441
Predicting: 2900/4441
Predicting: 3000/4441
Predicting: 3100/4441
Predicting: 3200/4441
Predicting: 3300/4441
Predicting: 3400/4441
Predicting: 3500/4441
Predicting: 3600/4441
Predicting: 3700/4441
Predicting: 3800/4441
Predicting: 3900/4441
Predicting: 4000/4441
Predicting: 4100/4441
Predicting: 4200/4441
Predicting: 4300/4441
Predicting: 4400/4441
Results saved to: /con

In [ ]:

# === Zip & Download: detected faces and metrics (original vs enhanced) ===
import os, zipfile, glob
from datetime import datetime

# Try to import Colab downloader if available
try:
    from google.colab import files as colab_files
except Exception:
    colab_files = None

def zip_dir(dir_path, zip_path):
    os.makedirs(os.path.dirname(zip_path), exist_ok=True)
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for root, _, files in os.walk(dir_path):
            for fname in files:
                fpath = os.path.join(root, fname)
                arcname = os.path.relpath(fpath, start=dir_path)
                zf.write(fpath, arcname)
    print(f"Created: {zip_path}")
    if colab_files:
        colab_files.download(zip_path)

def zip_files(file_paths, zip_path):
    os.makedirs(os.path.dirname(zip_path), exist_ok=True)
    with zipfile.ZipFile(zip_path, 'w', compression=zipfile.ZIP_DEFLATED) as zf:
        for f in file_paths:
            if os.path.exists(f):
                zf.write(f, os.path.basename(f))
            else:
                print(f"Warning: missing file not added: {f}")
    print(f"Created: {zip_path}")
    if colab_files:
        colab_files.download(zip_path)

timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Detected faces folders produced by the wrapper
detected_original = os.path.join(BASE_DIR, "detected_faces_original")
detected_enhanced = os.path.join(BASE_DIR, "detected_faces_enhanced")

detected_original_zip = os.path.join(BASE_DIR, f"detected_faces_original_{timestamp}.zip")
detected_enhanced_zip = os.path.join(BASE_DIR, f"detected_faces_enhanced_{timestamp}.zip")

if os.path.isdir(detected_original):
    zip_dir(detected_original, detected_original_zip)
else:
    print(f"Detected faces folder not found: {detected_original}")

if os.path.isdir(detected_enhanced):
    zip_dir(detected_enhanced, detected_enhanced_zip)
else:
    print(f"Detected faces folder not found: {detected_enhanced}")

# Metrics CSVs (separate zips for original vs enhanced)
metrics_original = [
    os.path.join(BASE_DIR, "test_outputs_original.csv"),
    os.path.join(BASE_DIR, "bias_p_metrics_original.csv"),
    os.path.join(BASE_DIR, "bias_w_metrics_original.csv"),
    os.path.join(BASE_DIR, "ens_original.csv"),
    os.path.join(BASE_DIR, "kl_divergence_metrics_original.csv"),
]
metrics_enhanced = [
    os.path.join(BASE_DIR, "test_outputs_enhanced.csv"),
    os.path.join(BASE_DIR, "bias_p_metrics_enhanced.csv"),
    os.path.join(BASE_DIR, "bias_w_metrics_enhanced.csv"),
    os.path.join(BASE_DIR, "ens_enhanced.csv"),
    os.path.join(BASE_DIR, "kl_divergence_metrics_enhanced.csv"),
]

# (Bias-W is intentionally NOT included)

metrics_original_zip = os.path.join(BASE_DIR, f"metrics_original_{timestamp}.zip")
metrics_enhanced_zip = os.path.join(BASE_DIR, f"metrics_enhanced_{timestamp}.zip")

zip_files(metrics_original, metrics_original_zip)
zip_files(metrics_enhanced, metrics_enhanced_zip)

print("Done preparing zips.")


Created: /content/metrics_results/detected_faces_original_20251028_191733.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Created: /content/metrics_results/detected_faces_enhanced_20251028_191733.zip


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

Created: /content/metrics_results/metrics_enhanced_20251028_191733.zip


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Done preparing zips.
